In [1]:
import numpy as np

def generation(n, alpha=2, beta=0, gamma=1, delta=0):
    y = [0]*n
    tan = np.tan(np.pi*alpha/2)
    S = (1 + beta**2 * tan**2) ** (1 / (2 * alpha))
    B = (1 / alpha) * np.arctan(beta * tan)
    print("S:", S, "B:", B, "tan(pi*alpha/2)", tan)

    for i in range(n):
        w = -np.log(1 - np.random.rand())
        u = np.random.uniform(-np.pi/2, np.pi/2)
        
        if alpha == 1:
            print("before transf:", y[i])
            y[i] = 2 / np.pi * ((np.pi/2 + beta * u) * np.tan(u) - beta * np.log((np.pi/2 * w * np.cos(u)) / (np.pi/2 + beta * u)))
            print("after transf:", y[i])
        else:
            print("before transf:", y[i])
            y[i] = S * np.sin(alpha) * (u + B) / (np.cos(u)**(1 / alpha)) * (np.cos(u - alpha * (u + B)) / w) ** ((1 - alpha) / alpha)
            print("after transf:", y[i])

        print("before scaling:", y[i])
        y[i] = gamma * y[i] + delta
        print("after scaling:", y[i])
    return y

$\theta = (\alpha, \beta, \gamma, \delta)$ 

N = 1000

$K_{\epsilon} (y-x)$ def by $S(y)$ ~ $N(S(x), \epsilon^2 \hat{\Sigma})$

$\hat{\Sigma}$ is an estimate of $Cov(S(x)\hat{\theta})$

$\Lambda$ = diag(0.25, 0.25, 1, 1)

In [ ]:
}

In [ ]:
{}